In [1]:
!pip install transformers
!pip install datasets

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

In [3]:
dataset= load_dataset("dair-ai/emotion")

README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [5]:
#check a sample record
dataset["train"][400]

{'text': 'i feel some of my projects are clever and useful enough i figured i would start sharing them on instructables so i wrote my first one this weekend',
 'label': 1}

In [6]:
def preprocess_function(examples):
    text=examples["text"]
    return tokenizer(text,padding="max_length",truncation=True)

In [7]:
model="bert-base-uncased"

In [8]:
tokenizer=AutoTokenizer.from_pretrained(model)
model=AutoModelForSequenceClassification.from_pretrained(model,num_labels=6)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
train_encode=dataset['train'].map(preprocess_function,batched=True)
validation_encode=dataset['validation'].map(preprocess_function,batched=True)
test_encode=dataset['test'].map(preprocess_function,batched=True)



Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [10]:
from transformers import TrainingArguments, Trainer


In [11]:
training_args=TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    save_steps=1000,  # Reduce checkpointing frequency, or
    save_total_limit=1  # Limit to 1 checkpoint (delete older ones)

)

In [12]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encode,
    eval_dataset=validation_encode,
)

In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,1.406300
1000,1.115700
1500,0.855600
2000,0.694300
2500,0.493900
3000,0.539300
3500,0.479200
4000,0.475200
4500,0.496500
5000,0.398700


TrainOutput(global_step=8000, training_loss=0.5502777242660523, metrics={'train_runtime': 1340.8035, 'train_samples_per_second': 11.933, 'train_steps_per_second': 5.967, 'total_flos': 4209928077312000.0, 'train_loss': 0.5502777242660523, 'epoch': 1.0})

In [ ]:
from sklearn.metrics import accuracy_score

predictions, labels, _ = trainer.predict(validation_encode)
accuracy = accuracy_score(labels, predictions.argmax(-1))
print(f"Accuracy on validation set: {accuracy:.4f}")